# **Lok Sabha Data Scraper**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
a=os.listdir("/content/drive/MyDrive/LOK SABHA DATA/05")
len(a)

In [ ]:
# module imports
!pip install tika
import pandas as pd
import os
import requests
import numpy as np
import multiprocessing
from tika import parser
from bs4 import BeautifulSoup
import re

  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32891 sha256=a2565a2bee797a704036100fe845a9aa2b01c98512ce54172a77b763ea024b0f
  Stored in directory: /root/.cache/pip/wheels/ec/2b/38/58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
Successfully built tika


### **Downloading all the pdfs from the Lok Sabha Digital Library**

In [ ]:
# a function to scrape the lok sabha digital library and download the pdfs

def scrape(path):

  # extracting folder name and creating a folder in drive
  folderName = path[-2:]

  if folderName in os.listdir("/content/drive/MyDrive/Lok Sabha Data"):
    print(folderName, " already present")
    
  else:
    folderPath = "/content/drive/MyDrive/Lok Sabha Data/" + folderName + "/"
    os.makedirs(folderPath)

    url = path
    links = []

    while url != None:
      req = requests.get(url)
      soup = BeautifulSoup(req.content, 'html.parser')

      # trying to fetch all the intermediate urls to the target pdfs
      table = soup.find('table', attrs = {'summary':'This table browses all dspace content'})
      for row in table.findAll('td', attrs = {'headers':'t4'}):
        if(row.a != None):
          link = row.a['href']
          links.append("https://eparlib.nic.in"+link)
      
      url = soup.find('a', attrs = {'class' : 'pull-right'})
      if url != None:
        url = "https://eparlib.nic.in" + url['href']
    
    # get to the pdf from here on
    for number,link in enumerate(links):
      req = requests.get(link)
      soup = BeautifulSoup(req.content, 'html.parser')

      table = soup.find('table', attrs = {'class' : 'table panel-body'})
      for row in table.findAll('td', attrs = {'align':'center'}):
        if(row.a != None):
          pdfLink = row.a['href']
          pdfLink = "https://eparlib.nic.in" + pdfLink

          # extracting and saving the pdfs
          try:
            pdfReq = requests.get(pdfLink)
            pdfName = str(number+1)
            pdfPath = folderPath + pdfName + ".pdf"
            with open(pdfPath, 'wb') as f:
              f.write(pdfReq.content)

          except requests.exceptions.HTTPError as errh:
              print("Http Error:",errh)
              print("Number : ", pdfName)
          except requests.exceptions.ConnectionError as errc:
              print("Error Connecting:",errc)
              print("Number : ", pdfName)
          except requests.exceptions.Timeout as errt:
              print("Timeout Error:",errt)
              print("Number : ", pdfName)
          except requests.exceptions.RequestException as err:
              print("OOps: Something Else",err)
              print("Number : ", pdfName)

    print("Done with ", folderName)



if __name__ == '__main__':
    
  pool = multiprocessing.Pool()
  pool = multiprocessing.Pool(processes=2)
  
  paths = [
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=01",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=02",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=03",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=04",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=05",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=06",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=07",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=08",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=09",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=10",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=11",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=12",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=13",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=14",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=15",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=16",
         "https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&sort_by=1&order=DESC&rpp=20&value=17"
  ]

  pool.map(scrape, paths)

01  already present
04  already present
02  already present
05  already present
03  already present
06  already present
07  already present
10  already present
08  already present
09  already present
11  already present
13  already present
12  already present
14  already present
15  already present
16  already present
17  already present


# **Reading PDFs to text and pre-processing text (Loksabha wise)**

## 17th

In [ ]:
def removeend(s):
    se=['.', '।','?','|', '!', '-']
    k=len(s)-1
    while k!=-1 and ((s[k] not in se) or (k!=0 and s[k]=='.' and s[k-1].isupper()==True)):
      #print(s[k])
      k = k-1
    #print(s[k])
    if len(s)-k>30:
      return s
    else:
      return s[:k+1]

def removestart(s):
    se={' ','-',':','\n','।'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

for fid in range(1,113):
  print(fid)
  raw = parser.from_file('/content/drive/MyDrive/LOK SABHA DATA/17/' + str(fid) + '.pdf')
  content = raw['content']

  startind = content.find('in the Chair]')     
  #startind = content.find('[MADAM SPEAKER in the Chair]')
  if startind==-1:
    startind=content.find('The Lok Sabha met at')
  if startind==-1:
    startind=0
    print('empty')

  content = content[startind+28:]
  cnt=0
  startindlist = []
  for match in re.finditer(r':\s*[^-]', content):
    startindlist.append(match.start())
    cnt+=1

  #print(cnt)
  #break
  startindlist.append(len(content))

  convlist =[]
  for i in range(len(startindlist)-1):
    convlist.append(content[startindlist[i]+1 : startindlist[i+1]])
  convlist1 =[]
  for j,i in enumerate(convlist):
    #print(j)

    i = re.sub('\(Interruptions\)', '', i)
    i = re.sub('\(/nterruptions\)', '', i)
    i = re.sub('(व्यवधान)', '', i)
    
    i = removeend(i)
    #break
    i = re.sub(r'\[*\(*Placed in Library,\s*See No.\s*LT\s*\d*\d\d\d\d/\d\d/\d\d\)*\]*', '', i)

    i = re.sub(r'\d\.\d\.20\d\d', '', i)
    i = re.sub(r'\d\d\.\d\d\.20\d\d', '', i)

    i = re.sub(r'\d\d\s*\d\d\s*hrs', '', i)
    
    i = re.sub(r'\n', '', i)
    i = re.sub(r' +', ' ', i)
    i = re.sub(r'\.+', ' ', i)
    i = re.sub(r'_+', ' ', i)
    i = re.sub(r'…', ' ', i)
    i = removestart(i)
    
    #print(len(i))
    if len(i)>0 and (i[-1]!='.' or i[-1]!='|' or i[-1]!='।') :
      if len(i)>1 and ord(i[-2])>2000 and ord(i[-2])<3000:
        i=i+'।'
      else:
        i = i + '.'
    #i = re.sub(y, ' ', i)
    convlist1.append(i)
  
  convlist6 = []
  ind =0
  for j,i in enumerate(convlist1):
    if len(i)!=0:
      convlist6.append(convlist1[j])
      ind+=1

  dframe = pd.DataFrame(convlist6)  
  #f=dframe.iloc[25]   
  #print(dframe)
  #dframe.to_csv('file_name')
  
  dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/17/'+str(fid)+ '.csv')

Streaming output truncated to the last 5000 lines.
1385
21
1405
16
349
597
16
551
1837
240
336
2711
2523
226
4899
7712
3797
3448
2231
48
1278
30
22
48
2564
147
35
4924
146
3863
144
361
906
938
1620
1100
828
98
71
62
103
45
127
73
1137
911
70
88
149
778
869
618
585
532
418
390
440
1305
572
1008
1364
228
222
1145
761
44
446
92
149
1129
262
289
143
1972
1306
776
126
1025
99
186
100
396
425
90
564
735
43
351
798
277
598
2069
38
138
85
1071
118
66
515
1164
982
1073
48
301
78
2681
895
61
379
77
21
85
20
109
73
181
655
563
46
2357
69
154
142
940
45
1127
1141
3185
296
764
851
626
675
125
323
420
426
120
249
51
347
755
95
165
48
509
0
565
44
165
48
475
0
466
107
108
26
22
19
41
121
38
72
296
375
0
339
152
128
124
125
15
189
19
26
524
116
60
126
56
45
98
2026
826
46
1144
0
339
51
228
163
0
85
33
104
183
246
1925
848
228
599
58
1298
52
70
574
100
31
104
350
335
86
397
298
575
1047
107
2280
81
64
12
0
85
58
171
132
0
78
74
778
1584
1706
0
78
60
160
129
0
73
79
2184
135
53
139
1386
38
140
204
2046


##16th

In [ ]:
def removeend(s):
    se=['.', '।','?','|', '!', '-','"']
    k=len(s)-1
    while k!=-1 and ((s[k] not in se) or (k!=0 and s[k]=='.' and s[k-1].isupper()==True)):
      #print(s[k])
      k = k-1
    #print(s[k])
    if len(s)-k>30:
      return s
    else:
      return s[:k+1]

def removestart(s):
    se={' ','-',':','\n','।'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

for fid in range(1,72):
  print(fid)
  raw = parser.from_file('/content/drive/MyDrive/LOK SABHA DATA/16/' + str(fid) + '.pdf')
  content = raw['content']

  startind = content.find('HON. S')        #[HON. SPEAKER in the Chair]       
  #startind = content.find('[MADAM SPEAKER in the Chair]')
  if startind==-1:
    startind=0
    print('empty')

  endind =  content.find('The Lok Sabha then adjourned')
  if endind == -1:
    endind = content.find('(The National Song was played')
  if endind == -1:
    endind = content.find('NATIONAL SONG')

  content = content[startind:endind]
  
  cnt=0
  startindlist = []
  for match in re.finditer(r':\s*[^-]', content):
    startindlist.append(match.start())
    cnt+=1

  #print(cnt)

  startindlist.append(len(content))

  convlist =[]
  for i in range(len(startindlist)-1):
    convlist.append(content[startindlist[i]+1 : startindlist[i+1]])
  convlist1 =[]
  for j,i in enumerate(convlist):
    #print(j)

    i = re.sub('\(Interruptions\)', '', i)
    i = re.sub('\(/nterruptions\)', '', i)
    i = re.sub('(व्यवधान)', '', i)
    
    #print(i)
    i = removeend(i)
    #break
    i = re.sub(r'\[*\(*Placed in Library,\s*See No.\s*LT\s*\d*\d\d\d\d/\d\d/\d\d\)*\]*', '', i)

    i = re.sub(r'\d\.\d\.20\d\d', '', i)
    i = re.sub(r'\d\d\.\d\d\.20\d\d', '', i)

    i = re.sub(r'\d\d\s*\d\d\s*hrs', '', i)
    
    i = re.sub(r'\n', '', i)
    i = re.sub(r' +', ' ', i)
    i = re.sub(r'_+', ' ', i)
    i = re.sub(r'…', ' ', i)
    i = re.sub(r'\.+', ' ', i)
    i = removestart(i)
  
    if len(i)!=0 and (i[-1]!='.' or i[-1]!='|' or i[-1]!='।'):
      if len(i)>1 and ord(i[-2])>2000 and ord(i[-2])<3000:
        i=i+'।'
      else:
        i = i + '.'

    #i = re.sub(y, ' ', i)
    convlist1.append(i)

  convlist6 = []
  ind =0
  for j,i in enumerate(convlist1):
    if len(i)!=0:
      convlist6.append(convlist1[j])
      ind+=1

  dframe = pd.DataFrame(convlist6)  
  #f=dframe.iloc[25]   
  #print(dframe)
  #dframe.to_csv('file_name')
  
  dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/16/'+str(fid)+ '.csv')

## 15th

### Senket's converter

In [ ]:
#  Vowels

vowels = {
    "Â" : b"\xe0\xa5\x8d",       # aadha
    # b"" : b"",       # 
    # b"" : b"",       # au
    # b"" : b"",       # ooo
    "+ÉÉä" : b"\xe0\xa4\x93",       # o
    # b"" : b"\xe0\xa4\x90",       # aee
    "+ÉÉì" : b"\xe0\xa4\x91",       # aw
    "+ÉÉè" : b"\xe0\xa4\x94",      # aau
    "Aä" : b"\xe0\xa4\x8e",       # ae
    ">ó" : b"\xe0\xa4\x8a",       # uuu
    "<Ç" : b"\xe0\xa4\x88",       # eee
    "+ÉÉ" : b"\xe0\xa4\x86",       # aa
    "+É" : b"\xe0\xa4\x85",       # a
}

# halfVowels with extra care

vowelsExtraCare = {
    "*" : b"\xe0\xa5\xa4",       # purna-viraam
    "A" : b"\xe0\xa4\x8f",       # ai
    ">" : b"\xe0\xa4\x8a",       # uuu
    "=" : b"\xe0\xa4\x89",       # uu
    "<" : b"\xe0\xa4\x87",       # ee
}

In [ ]:
# consonants with extra care

consonantsExtraCare = {
    "R" : b"\xe0\xa4\x99",       # nga
    "c" : b"\xe0\xa4\xb9",       # ha
    "U" : b"\xe0\xa4\x9b",       # chha
    "]" : b"\xe0\xa4\x9f",       # tta
    "~" : b"\xe0\xa4\xa0",       # ttha
    "b" : b"\xe0\xa4\xa1",       # dda
    "f" : b"\xe0\xa4\xa2",       # ddha
    "n" : b"\xe0\xa4\xa6",       # da
}

# Aryan2 to Unicode Map for consonants

consonants = {
    "ÉÌBÉE" : b"\xe0\xa4\xb0" + vowels["Â"] + b"\xe0\xa4\x95" + b"\xe0\xa4\xbf",       # kri
    "#ÉEÉ" : b"\xe0\xa4\x95\xe0\xa5\x8d\xe0\xa4\xb0\xe0\xa4\xbe",       # kra
    "BÉäE" : b"\xe0\xa4\x95\xe0\xa5\x87",       # ke
    "BÉßE" : b"\xe0\xa4\x95\xe0\xa5\x83",       # kru
    "BÉÖE" : b"\xe0\xa4\x95\xe0\xa5\x81",       # ku
    "BÉE" : b"\xe0\xa4\x95",       # ka
    "JÉ" : b"\xe0\xa4\x96",       #kha
    "MÉ" : b"\xe0\xa4\x97",       # ga
    "PÉ" : b"\xe0\xa4\x98",       # gha
    "SÉ" : b"\xe0\xa4\x9a",       # cha
    "VÉ" : b"\xe0\xa4\x9c",       # ja
    "ZÉ" : b"\xe0\xa4\x9d",       # jha
    # b"" : b"",       # nya
    "½" : b"\xe0\xa5\x9c",      # ada
    "ªÉ" : b"\xe0\xa4\xaf",       # ya
    "¶É" : b"\xe0\xa4\xb6",       # sha
    "hÉ" : b"\xe0\xa4\xa3",       # nna
    "®️" : b"\xe0\xa4\xb0",       # ra
    "−É" : b"\xe0\xa4\xb7",       # ssa
    "iÉ" : b"\xe0\xa4\xa4",       # ta
    "lÉ" : b"\xe0\xa4\xa5",       # tha
    "vÉ" : b"\xe0\xa4\xa7",       # dha
    "xÉ" : b"\xe0\xa4\xa8",       # na
    "ãÉ" : b"\xe0\xa4\xb2",      # la
    "ºÉ" : b"\xe0\xa4\xb8",       # sa
    "{ÉE" : b"\xe0\xa4\xab",       # fa
    "{ÉD" : b"\xe0\xa4\xab" + vowels["Â"],       # f
    "{É" : b"\xe0\xa4\xaa",      # pa
    "¤É" : b"\xe0\xa4\xac",       # ba
    "£É" : b"\xe0\xa4\xad",       # bha
    "àÉ" : b"\xe0\xa4\xae",       # ma
    "´É" : b"\xe0\xa4\xb5",       # va
    "bÅ" : consonantsExtraCare["b"] + vowels["Â"] + b"\xe0\xa4\xb0",       # ddra
    "]Å" : b"\xe0\xa4\x9f" + vowels["Â"] + b"\xe0\xa4\xb0",       # ttra
}

# Alag se aadha

halfConsonants = {
    "BÉD" : consonants["BÉE"] + vowels["Â"],       # k
    "ª" : b"\xe0\xa4\xaf" + vowels["Â"],       # y
    "¶" : b"\xe0\xa4\xb6" + vowels["Â"],       # sh
    "−" : b"\xe0\xa4\xb7" + vowels["Â"],       # ss
    "ã" : b"\xe0\xa4\xb2" + vowels["Â"],      # l
    "º" : b"\xe0\xa4\xb8" + vowels["Â"],       # s
    "¤" : b"\xe0\xa4\xac" + vowels["Â"],       # b
    "£" : b"\xe0\xa4\xad" + vowels["Â"],       # bh
    "à" : b"\xe0\xa4\xae" + vowels["Â"],       # m
    "´" : b"\xe0\xa4\xb5" + vowels["Â"],       # v
    "B" : b"\xe0\xa4\xb5" + vowels["Â"],       # v
}

# halfConsonants with extra care

halfConsonantsExtraCare = {
    "J" : b"\xe0\xa4\x96" + vowels["Â"],       # kh
    "M" : b"\xe0\xa4\x97" + vowels["Â"],       # g
    "P" : b"\xe0\xa4\x98" + vowels["Â"],       # gh
    "S" : b"\xe0\xa4\x9a" + vowels["Â"],       # ch
    "V" : b"\xe0\xa4\x9c" + vowels["Â"],       # j
    "Z" : b"\xe0\xa4\x9d" + vowels["Â"],       # jh
    "i" : b"\xe0\xa4\xa4" + vowels["Â"],       # t
    "l" : b"\xe0\xa4\xa5" + vowels["Â"],       # th
    "v" : b"\xe0\xa4\xa7" + vowels["Â"],       # dh
    "x" : b"\xe0\xa4\xa8" + vowels["Â"],       # n
    "{" : b"\xe0\xa4\xaa" + vowels["Â"],      # p
    "h" : b"\xe0\xa4\xa3" + vowels["Â"],       # nn
}

In [ ]:
# baadme aane wali maatra

maatrasPiche = {
    "Ã" : b"\xe0\xa4\xbc",       # nukta
    "Þ" : b'\x22',       # double quote
    "&" : b"\xe0\xa4\x83",       #aha
    "ì" : b"\xe0\xa5\x85",       # chandra
    "é" : b"\xe0\xa5\x88\xe0\xa4\x82",       # aain
    "Ú" : b"\xe0\xa5\x82",       # bada uu
    "Ö" : b"\xe0\xa5\x81",       # chota uu
    "Éå" : b"\xe0\xa5\x8b\xe0\xa4\x82",       # on
    "ÉÒ" : b"\xe0\xa5\x80",       # badi ee
    "ÉÓ" : b"\xe0\xa5\x80",       # badi ee
    "Éä" : b"\xe0\xa5\x8b",       # O
    "è" : b"\xe0\xa5\x88",       # au
    "ä" : b"\xe0\xa5\x87",       # ae
    "É" : b"\xe0\xa4\xbe",       # aa danda
    "å" : b"\xe0\xa5\x87\xe0\xa4\x82",       # aen
    "Æ" : b"\xe0\xa4\x82",       # anusvaraa
    "ß" : b"\xe0\xa5\x83",       # niche wali ra ki maatra
}

# pehle aane wali maatra

maatrasAage = {
    "ÉÊ" : b"\xe0\xa4\xbf",       # choti ee
    "ÉÎ" : b"\xe0\xa4\xbf",       # choti ee
    "ÉË" : b"\xe0\xa4\xbf" + maatrasPiche["Æ"],       # ein
    "ÉÌ" : b"\xe0\xa4\xbf" + consonants["®️"] + vowels["Â"],       # choti ee + upar wali ra ki maatra
}

# jo pehle aani chahiye but baadme aayi hain

maatrasPicheAage = {
    "Ç" : b"\xe0\xa4\xb0\xe0\xa5\x8d",       # upar wali ra ki maatra
}

In [ ]:
# Conjuncts

conjucts = {
    "kÉ" : consonants["iÉ"] + vowels["Â"] + consonants["iÉ"],       # aadha ta + ta
    "µÉE" : b"\xe0\xa4\x95" + vowels["Â"] + b"\xe0\xa4\xb0",       # kra
    "FÉ" : consonants["BÉE"] + vowels["Â"] + consonants["−É"],      # ksha
    "|É" : consonants["{É"] + vowels["Â"] + consonants["®️"],       # pra
    "jÉ" : consonants["iÉ"] + vowels["Â"] + consonants["®️"],       # tra
    "°ô" : consonants["®️"] + maatrasPiche["Ú"],       # ruu
    "gÉ" : consonants["¶É"] + vowels["Â"] + consonants["®️"],       # shra
    "OÉ" : consonants["MÉ"] + vowels["Â"] + consonants["®️"],       # gra
    "tÉ" : consonantsExtraCare["n"] + vowels["Â"] + consonants["ªÉ"],       # dya
    "âó" : consonants["®️"] + maatrasPiche["Ö"],       # ru
    "QÉ" : consonants["PÉ"] + vowels["Â"] + consonants["®️"],       # ghra
    "¥É" : consonants["¤É"] + vowels["Â"] + consonants["®️"],       # bra
    "©️É" : consonants["àÉ"] + vowels["Â"] + consonants["®️"],       # mra
}

# extra care conjucts

conjuctsExtraCare = {
    "r" : consonantsExtraCare["n"] + vowels["Â"] + consonants["vÉ"],       # da + dha
    "p" : consonantsExtraCare["n"] + vowels["Â"] + consonants["®️"],       # dra
    "u" : consonantsExtraCare["n"] + vowels["Â"] + consonants["´É"],       # dva
}

In [ ]:
def mapUnicode(x):

  for iter in range(0,3):

    # replacing consonants
    for key in consonants:
      x = x.replace(key, consonants[key].decode('utf-8'))
    
    # replacing consonants extra care
    for key in consonantsExtraCare:
      indexes = []
      for i in range(0, len(x)):
        if (x[i] == key):
          count = 0
          for j in range(1,4):
            if (i-j>-1 and ord(x[i-j])>2000  and ord(x[i-j])<4000):
              count += 1
            if (i+j<len(x) and ord(x[i+j])>2000 and ord(x[i+j])<4000):
              count += 1
          if count>0:
            indexes.append(i)
    
      for index in indexes:
        x = x[:index] + consonantsExtraCare[key].decode('utf-8') + x[(index+1):]
    
    # replacing conjucts
    for key in conjucts:
      x = x.replace(key, conjucts[key].decode('utf-8'))
    
    # replacing conjucts extra care
    for key in conjuctsExtraCare:
      indexes = []

      for i in range(0, len(x)):
        if x[i]==key:
          if ((i>0 and ord(x[i-1])>400) or (i<len(x)-1 and ord(x[i+1])>400)):
            x = x[:i] + conjuctsExtraCare[key].decode('utf-8') + x[(i+1):]
    
    # replacing half consonants
    for key in halfConsonants:
      x = x.replace(key, halfConsonants[key].decode('utf-8'))
    
    # replacing half consonants extra care
    for key in halfConsonantsExtraCare:
      indexes = []

      for i in range(0, len(x)):
        if x[i]==key:
          if ((i>0 and ord(x[i-1])>400) or (i<len(x)-1 and ord(x[i+1])>400)):
            x = x[:i] + halfConsonantsExtraCare[key].decode('utf-8') + x[(i+1):]
    
    # replacing maatras aage
    for key in maatrasAage:
      indexes = []

      for m in re.finditer(key, x):
        indexes.append(m.start())
      
      for index in indexes:
        if (index+3<len(x) and x[index+3]=='्'):
          x = x[:index] + x[(index+2):(index+5)] + key + x[(index+5):]
        else:
          x = x[:index] + x[index+len(key)] + key + x[(index+len(key)+1):]
        
      x = x.replace(key, maatrasAage[key].decode('utf-8'))
    
    # replacing vowels
    for key in vowels:
      x = x.replace(key, vowels[key].decode('utf-8'))
    
    # replacing vowels extra care
    for key in vowelsExtraCare:
      indexes = []
      for i in range(0, len(x)):
        if (x[i] == key):
          count = 0
          for j in range(1,5):
            if (i-j>-1 and ord(x[i-j])>2000):
              count += 1
            if (i+j<len(x) and ord(x[i+j])>2000):
              count += 1
          if count>0:
            indexes.append(i)
      
      for index in indexes:
        x = x[:index] + vowelsExtraCare[key].decode('utf-8') + x[(index+1):]
    
    # replacing maatras piche
    for key in maatrasPiche:
      x = x.replace(key, maatrasPiche[key].decode('utf-8'))
    
    # replacing aage aane wali piche hokar maatras
    for key in maatrasPicheAage:
      indexes = []

      for m in re.finditer(key, x):
        indexes.append(m.start())
      
      for index in indexes:
        x = x[:(index-1)] + key + x[index-1] + x[(index+1):]
        
      x = x.replace(key, maatrasPicheAage[key].decode('utf-8'))

  return x

### Code

In [ ]:
def removeend(s):
    se=['.', '।','?','|', '!', '-','"']
    k=len(s)-1
    while k!=-1 and ((s[k] not in se) or (k!=0 and s[k]=='.' and s[k-1].isupper()==True)):
      #print(s[k])
      k = k-1
    #print(s[k])
    if len(s)-k>30:
      return s
    else:
      return s[:k+1]

def removestart(s):
    se={' ','-',':','\n'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

for fid in range(1,358):
  print(fid)
  raw = parser.from_file('/content/drive/MyDrive/Lok Sabha Data/PDFs/15/' + str(fid) + '.pdf')
  content = raw['content']

  startind = content.find('[MADAM SPEAKER in the Chair]')
  if startind==-1:
    startind=content.find('The Lok Sabha met at')        
  if startind==-1:
    startind=content.find('OBITUARY REFERENCE') 
  if startind==-1:
    startind=content.find('LOK SABHA DEBATES') 
  if startind==-1:
    startind=content.find('LOK SABHA') 
  if startind==-1:
    print("empty")
    startind = 0

  # endind =  content.find('The Lok Sabha then adjourned')
  # if endind == -1:
  #   endind = content.find('(The National Song was played')
  # if endind == -1:
  #   endind = content.find('NATIONAL SONG')

  content = content[startind:]
  cnt=0
  startindlist = []

  p = re.findall(r':[^-]', content)
  #print(p)
  for match in re.finditer(r':[^-]', content):
    startindlist.append(match.start())
    cnt+=1

  #print(cnt)

  startindlist.append(len(content))

  convlist =[]
  for i in range(len(startindlist)-1):
    convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

  #print(convlist)
  convlist1 =[]
  for j,i in enumerate(convlist):
    i = mapUnicode(i)

    i = re.sub('\(Interruptions\)', '', i)
    i = re.sub('\(/nterruptions\)', '', i)
    i = re.sub('(व्यवधान)', '', i)
    
    #print(i)
    i = removeend(i)
    #break
    i = re.sub(r'\[*\(*Placed in Library,\s*See No.\s*LT\s*\d*\d\d\d\d/\d\d/\d\d\)*\]*', '', i)

    i = re.sub(r'\d\.\d\.20\d\d', '', i)
    i = re.sub(r'\d\d\.\d\d\.20\d\d', '', i)

    i = re.sub(r'\d\d\s*\d\d\s*hrs', '', i)
    
    i = re.sub(r'\n', '', i)
    i = re.sub(r' +', ' ', i)
    i = re.sub(r'_+', ' ', i)
    i = re.sub(r'…', ' ', i)
    i = re.sub(r'\.+', ' ', i)
    i = removestart(i)
  
    if len(i)!=0 and (i[-1]!='.' or i[-1]!='|' or i[-1]!='।'):
      if len(i)>1 and ord(i[-2])>2000 and ord(i[-2])<3000:
        i=i+'।'
      else:
        i = i + '.'

    #i = re.sub(y, ' ', i)
    convlist1.append(i)

  convlist6 = []
  ind =0
  for j,i in enumerate(convlist1):
    if len(i)!=0:
      convlist6.append(convlist1[j])
      ind+=1
  
  if ind==0:
    print("blank csvs")

  print(ind)
  print("\n")
  
  dframe = pd.DataFrame(convlist6)  

  dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/15/'+str(fid)+ '.csv')

1
383


2
304


3
318


4
351


5
196


6
23


7
61


8
59


9
51


10
50


11
55


12
63


13
205


14
81


15
2


16
66


17
87


18
58


19
41


20
78


21
25


22
3


23
900


24
841


25
665


26
95


27
34


28
91


29
1214


30
620


31
1220


32
442


33
42


34
63


35
115


36
2


37
159


38
178


39
99


40
121


41
134


42
138


43
113


44
47


45
44


46
79


47
37


48
54


49
322


50
68


51
67


52
2


53
40


54
62


55
51


56
80


57
76


58
415


59
193


60
387


61
608


62
507


63
81


64
561


65
497


66
521


67
479


68
47


69
73


70
2


71
31


72
351


73
343


74
286


75
18


76
225


77
82


78
878


79
92


80
388


81
213


82
138


83
139


84
324


85
312


86
503


87
102


88
726


89
514


90
2


91
217


92
327


93
76


94
50


95
26


96
87


97
51


98
43


99
46


100
49


101
124


102
23


103
63


104
51


105
71


106
34


107
38


108
49


109
33


110
304


111
2


112
200


113
110


114
462


115
416


116
263


117
482


118
2

## 14th

In [ ]:
def removeend(s):
    se={'(',')','.','\n',' '}
    k=len(s)-1
    while k!=-1 and (s[k].isupper()==True or s[k] in se):
      if s[k]==')':
        while k!=-1 and s[k]!='(':
          k=k-1
      else:
        k=k-1
    #print(s[i])
    return s[:k+1]

def removestart(s):
    se={' ','-',':','\n'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
l=[22,33,54,243]
#for fid in range(61,333):
for fid in l:
  print(fid)
  raw = parser.from_file('/content/drive/MyDrive/Lok Sabha Data/PDFs/14/' + str(fid) + '.pdf')
  content = raw['content']

  startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
  if startind==-1:
    startind = content.find('MR. SPeAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER ;n the')
  if startind==-1:
    startind = content.find('MR. DEPUTY SPEAKER in the')
  if startind==-1:
    startind=content.find('OBITUARY REFERENCE')  
  if startind==-1:
    startind=content.find('The Lok Sabha met at') 
  if startind==-1:
    startind = content.find('MR. S')
  if startind==-1:
    startind = content.find('Mr. S')
  if startind==-1:
    startind = content.find('Mr, S')  
  #if startind==-1:
    #startind = content.find('[S hri C hitta Basu in the')    
  if startind==-1:
    print("empty")
    #empty.append(fid)
    startind = 0
    
  
  #print(startind)
  endind = content.find('The Lok Sabha then adjourned')

  if endind<startind and endind!=-1:
    endind=-1
    startind=0
    print("emptyy")

  content = content[startind:endind]

  cnt=0
  startindlist = []
  for match in re.finditer(r'[A-Z)] *:', content):
    startindlist.append(match.start())
    cnt+=1

  convlist =[]
  for i in range(len(startindlist)-1):
    convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

  convlist1 =[]
  for j,i in enumerate(convlist):
    #print(j)

    i = re.sub('[\(\{\[]I?i?l?nterruptions[\)\}\]]', '', i)
    
    i = removeend(i)

    i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

    i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)
    i = re.sub(r'\d\d\s*\d\d\s*hrs', '', i)

    i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
    i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
    i = re.sub(r'ORAL ANSWERS', ' ', i)
    i = re.sub(r'Oral Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)

    i = re.sub(r'\n', '', i)
    i = re.sub(r' +', ' ', i)
    i = re.sub(r'_+', ' ', i)
    i = re.sub(r'…', ' ', i)
    i = re.sub(r'\.+', ' ', i)

    i = removestart(i)
    if len(i)!=0 and i[-1]!='.':
      i = i + '.'

    if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
    convlist1.append(i)

  convlist6 = []
  ind =0
  for j,i in enumerate(convlist1):
    if len(i)!=0:
      convlist6.append(convlist1[j])
      ind+=1

  if ind==0:
    print("blank csvs")

  dframe = pd.DataFrame(convlist6)  
  #print(dframe)   

  #dframe.to_csv('file_name1.csv')
  dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/14/'+str(fid)+ '.csv')

22
33
54
243


## 13th 12th

In [ ]:
def removeend(s):
    se={'(',')','.','\n',' ','\t'}
    k=len(s)-1
    while k!=-1 and (s[k].isupper()==True or s[k] in se):
      if s[k]==')':
        while k!=-1 and s[k]!='(':
          k=k-1
      else:
        k=k-1
    #print(s[i])
    return s[:k+1]

def removestart(s):
    se={' ','-',':','\n'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(1,358):
  print(fid)
  raw = parser.from_file('/content/drive/MyDrive/Lok Sabha Data/PDFs/13/' + str(fid) + '.pdf')
  content = raw['content']

  startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
  if startind==-1:
    startind = content.find('MR. SPeAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER ;n the')
  if startind==-1:
    startind = content.find('MR. DEPUTY SPEAKER in the')
  if startind==-1:
    startind=content.find('OBITUARY REFERENCE')  
  if startind==-1:
    startind=content.find('The Lok Sabha met at') 
  if startind==-1:
    startind = content.find('MR. S')
  if startind==-1:
    startind = content.find('MR, S')
  if startind==-1:
    startind = content.find('Mr. S')
  if startind==-1:
    startind = content.find('Mr, S')  
  #if startind==-1:
    #startind = content.find('[S hri C hitta Basu in the')    
  if startind==-1:
    print("empty")
    #empty.append(fid)
    startind = 0
  
  
  endind = content.find('The Lok Sabha then adjourned')

  if endind<startind and endind!=-1:
    endind=-1
    startind=0
    print("emptyy")

  content = content[startind:endind]

  cnt=0
  startindlist = []
  for match in re.finditer(r'[A-Z)] *:', content):
    startindlist.append(match.start())
    cnt+=1

  convlist =[]
  for i in range(len(startindlist)-1):
    convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

  convlist1 =[]
  for j,i in enumerate(convlist):
    #print(j)

    i = re.sub('[\(\{\[]I?i?l?nterruptions[\)\}\]]', '', i)
    
    i = removeend(i)

    i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

    i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)
    i = re.sub(r'\d\d\s*\d\d\s*hrs', '', i)

    i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
    i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
    i = re.sub(r'ORAL ANSWERS', ' ', i)
    i = re.sub(r'Oral Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)

    i = re.sub(r'\n', '', i)
    i = re.sub(r' +', ' ', i)
    i = re.sub(r'_+', ' ', i)
    i = re.sub(r'…', ' ', i)
    i = re.sub(r'\.+', ' ', i)

    i = removestart(i)
    if len(i)!=0 and i[-1]!='.':
      i = i + '.'

    if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
    convlist1.append(i)

  convlist6 = []
  ind =0
  for j,i in enumerate(convlist1):
    if len(i)!=0:
      convlist6.append(convlist1[j])
      ind+=1

  if ind==0:
    print("blank csvs")

  dframe = pd.DataFrame(convlist6)  
  #print(dframe)   

  #dframe.to_csv('file_name1.csv')
  dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/13/'+str(fid)+ '.csv')

1
emptyy
2
3
4
5
6
7
8
9
10
11
12
13
14
emptyy
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
emptyy
64
65
66
emptyy
67
68
69
70
71
72
emptyy
73
74
75
76
77
78
emptyy
79
80
81
82
83
84
85
blank csvs
86
emptyy
87
emptyy
88
89
90
91
92
93
94
95
96
97
98
emptyy
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
empty
145
146
147
148
emptyy
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
emptyy
164
165
166
167
168
169
170
171
172
173
174
175
176
emptyy
177
178
179
180
181
182
183
184
185
186
187
emptyy
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
emptyy
221
222
223
224
225
226
227
228
229
230
empty
blank csvs
231
blank csvs
232
233
234
235
236
237
238
239
240
241
2

In [ ]:
def removeend(s):
    se={'(',')','.','\n',' ','\t'}
    k=len(s)-1
    while k!=-1 and (s[k].isupper()==True or s[k] in se):
      if s[k]==')':
        while k!=-1 and s[k]!='(':
          k=k-1
      else:
        k=k-1
    #print(s[i])
    return s[:k+1]

def removestart(s):
    se={' ','-',':','\n'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(1,89):
  print(fid)
  raw = parser.from_file('/content/drive/MyDrive/Lok Sabha Data/PDFs/12/' + str(fid) + '.pdf')
  content = raw['content']

  startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
  if startind==-1:
    startind = content.find('MR. SPeAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER ;n the')
  if startind==-1:
    startind = content.find('MR. DEPUTY SPEAKER in the')
  if startind==-1:
    startind=content.find('OBITUARY REFERENCE')  
  if startind==-1:
    startind=content.find('The Lok Sabha met at') 
  if startind==-1:
    startind = content.find('MR. S')
  if startind==-1:
    startind = content.find('MR, S')
  if startind==-1:
    startind = content.find('Mr. S')
  if startind==-1:
    startind = content.find('Mr, S')  
  #if startind==-1:
    #startind = content.find('[S hri C hitta Basu in the')    
  if startind==-1:
    print("empty")
    #empty.append(fid)
    startind = 0
  
  #print(startind)
  endind = content.find('The Lok Sabha then adjourned')

  if endind<startind and endind!=-1:
    endind=-1
    startind=0
    print("emptyy")

  content = content[startind:endind]

  cnt=0
  startindlist = []
  for match in re.finditer(r'[A-Z)] *:', content):
    startindlist.append(match.start())
    cnt+=1

  convlist =[]
  for i in range(len(startindlist)-1):
    convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

  convlist1 =[]
  for j,i in enumerate(convlist):
    #print(j)

    i = re.sub('[\(\{\[]I?i?l?nterruptions[\)\}\]]', '', i)
    
    i = removeend(i)

    i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

    i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)
    i = re.sub(r'\d\d\s*\d\d\s*hrs', '', i)

    i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
    i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
    i = re.sub(r'ORAL ANSWERS', ' ', i)
    i = re.sub(r'Oral Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)

    i = re.sub(r'\n', '', i)
    i = re.sub(r' +', ' ', i)
    i = re.sub(r'_+', ' ', i)
    i = re.sub(r'…', ' ', i)
    i = re.sub(r'\.+', ' ', i)

    i = removestart(i)
    if len(i)!=0 and i[-1]!='.':
      i = i + '.'

    if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
    convlist1.append(i)

  convlist6 = []
  ind =0
  for j,i in enumerate(convlist1):
    if len(i)!=0:
      convlist6.append(convlist1[j])
      ind+=1

  if ind==0:
    print("blank csvs")

  dframe = pd.DataFrame(convlist6)  
  #print(dframe)   

  #dframe.to_csv('file_name1.csv')
  dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/12/'+str(fid)+ '.csv')

1
2
3
4
5
6
7
8
9
emptyy
10
11
12
13
14
15
16
17
18
19
20
21
22
emptyy
23
24
25
26
27
28
29
30
31
32
emptyy
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
emptyy
51
52
53
54
55
56
57
58
59
60
emptyy
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
emptyy
84
85
86
87
88


## 11th

In [ ]:
def removeend(s):
    se={'(',')','.','\n',' ','\t'}
    k=len(s)-1
    while k!=-1 and (s[k].isupper()==True or s[k] in se):
      if s[k]==')':
        while k!=-1 and s[k]!='(':
          k=k-1
      else:
        k=k-1
    #print(s[i])
    return s[:k+1]

def removestart(s):
    se={' ','-',':','\n'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(1,126):
  print(fid)
  raw = parser.from_file('/content/drive/MyDrive/Lok Sabha Data/PDFs/11/' + str(fid) + '.pdf')
  content = raw['content']

  startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
  if startind==-1:
    startind = content.find('MR. SPeAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER ;n the')
  if startind==-1:
    startind = content.find('MR. DEPUTY SPEAKER in the')
  if startind==-1:
    startind=content.find('OBITUARY REFERENCE')  
  if startind==-1:
    startind=content.find('The Lok Sabha met at') 
  if startind==-1:
    startind = content.find('MR. S')
  if startind==-1:
    startind = content.find('MR, S')
  if startind==-1:
    startind = content.find('Mr. S')
  if startind==-1:
    startind = content.find('Mr, S')  
  #if startind==-1:
    #startind = content.find('[S hri C hitta Basu in the')    
  if startind==-1:
    print("empty")
    #empty.append(fid)
    startind = 0
  
  #print(startind)
  endind = content.find('The Lok Sabha then adjourned')

  if endind<startind and endind!=-1:
    endind=-1
    startind=0
    print("emptyy")

  content = content[startind:endind]

  cnt=0
  startindlist = []
  for match in re.finditer(r'[A-Z)] *:', content):
    startindlist.append(match.start())
    cnt+=1

  convlist =[]
  for i in range(len(startindlist)-1):
    convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

  convlist1 =[]
  for j,i in enumerate(convlist):
    #print(j)

    i = re.sub('[\(\{\[]I?i?l?nterruptions[\)\}\]]', '', i)
    
    i = removeend(i)

    i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

    i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)
    i = re.sub(r'\d\d\s*\d\d\s*hrs', '', i)

    i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
    i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
    i = re.sub(r'ORAL ANSWERS', ' ', i)
    i = re.sub(r'Oral Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)

    i = re.sub(r'\n', '', i)
    i = re.sub(r' +', ' ', i)
    i = re.sub(r'_+', ' ', i)
    i = re.sub(r'…', ' ', i)
    i = re.sub(r'\.+', ' ', i)

    i = removestart(i)
    if len(i)!=0 and i[-1]!='.':
      i = i + '.'

    if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
    convlist1.append(i)

  convlist6 = []
  ind =0
  for j,i in enumerate(convlist1):
    if len(i)!=0:
      convlist6.append(convlist1[j])
      ind+=1

  if ind==0:
    print("blank csvs")

  dframe = pd.DataFrame(convlist6)  
  #print(dframe)   

  #dframe.to_csv('file_name1.csv')
  dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/11/'+str(fid)+ '.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
empty
26
27
28
29
30
31
32
emptyy
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
empty
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
emptyy
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125


## 10th

In [ ]:
def removeend(s):
    se={'(',')','.','\n',' ','\t'}
    k=len(s)-1
    while k!=-1 and (s[k].isupper()==True or s[k] in se):
      if s[k]==')':
        while k!=-1 and s[k]!='(':
          k=k-1
      else:
        k=k-1
    #print(s[i])
    return s[:k+1]

def removestart(s):
    se={' ','-',':','\n'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(423,424):
  print(fid)
  raw = parser.from_file('/content/drive/MyDrive/Lok Sabha Data/PDFs/10/' + str(fid) + '.pdf')
  content = raw['content']

  startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
  if startind==-1:
    startind = content.find('MR. SPeAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER ;n the')
  if startind==-1:
    startind = content.find('MR. DEPUTY SPEAKER in the')
  if startind==-1:
    startind=content.find('OBITUARY REFERENCE')  
  if startind==-1:
    startind=content.find('The Lok Sabha met at') 
  if startind==-1:
    startind = content.find('MR. S')
  if startind==-1:
    startind = content.find('MR, S')
  if startind==-1:
    startind = content.find('Mr. S')
  if startind==-1:
    startind = content.find('Mr, S')  
  #if startind==-1:
    #startind = content.find('[S hri C hitta Basu in the')    
  if startind==-1:
    print("empty")
    #empty.append(fid)
    startind = 0
  
  #print(startind)
  endind = content.find('The Lok Sabha then adjourned')

  if endind<startind and endind!=-1:
    endind=-1
    startind=0
    print("emptyy")

  content = content[startind:endind]

  cnt=0
  startindlist = []
  for match in re.finditer(r'[A-Z)] *:', content):
    startindlist.append(match.start())
    cnt+=1

  convlist =[]
  for i in range(len(startindlist)-1):
    convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

  convlist1 =[]
  for j,i in enumerate(convlist):
    #print(j)

    i = re.sub('[\(\{\[]I?i?l?nterruptions[\)\}\]]', '', i)
    
    i = removeend(i)

    i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

    i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)
    i = re.sub(r'\d\d\s*\d\d\s*hrs', '', i)

    i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
    i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
    i = re.sub(r'ORAL ANSWERS', ' ', i)
    i = re.sub(r'Oral Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)

    i = re.sub(r'\n', '', i)
    i = re.sub(r' +', ' ', i)
    i = re.sub(r'_+', ' ', i)
    i = re.sub(r'…', ' ', i)
    i = re.sub(r'\.+', ' ', i)

    i = removestart(i)
    if len(i)!=0 and i[-1]!='.':
      i = i + '.'

    if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
    convlist1.append(i)

  convlist6 = []
  ind =0
  for j,i in enumerate(convlist1):
    if len(i)!=0:
      convlist6.append(convlist1[j])
      ind+=1

  if ind==0:
    print("blank csvs")

  dframe = pd.DataFrame(convlist6)  
  #print(dframe)   

  #dframe.to_csv('file_name1.csv')
  dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/10/'+str(fid)+ '.csv')

423


2021-10-20 11:03:42,281 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2021-10-20 11:03:42,903 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2021-10-20 11:03:43,298 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


## 9th

In [ ]:
def removeend(s):
    se={'(',')','.','\n',' ','\t'}
    k=len(s)-1
    while k!=-1 and (s[k].isupper()==True or s[k] in se):
      if s[k]==')':
        while k!=-1 and s[k]!='(':
          k=k-1
      else:
        k=k-1
    #print(s[i])
    return s[:k+1]

def removestart(s):
    se={' ','-',':','\n'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(50,51):
  print(fid)
  raw = parser.from_file('/content/drive/MyDrive/Lok Sabha Data/PDFs/09/' + str(fid) + '.pdf')
  content = raw['content']

  startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
  if startind==-1:
    startind = content.find('MR. SPeAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER ;n the')
  if startind==-1:
    startind = content.find('MR. DEPUTY SPEAKER in the')
  if startind==-1:
    startind=content.find('OBITUARY REFERENCE')  
  if startind==-1:
    startind=content.find('The Lok Sabha met at') 
  if startind==-1:
    startind = content.find('MR. S')
  if startind==-1:
    startind = content.find('MR, S')
  if startind==-1:
    startind = content.find('Mr. S')
  if startind==-1:
    startind = content.find('Mr, S')  
  #if startind==-1:
    #startind = content.find('[S hri C hitta Basu in the')    
  if startind==-1:
    print("empty")
    #empty.append(fid)
    startind = 0
  
  #print(startind)
  endind = content.find('The Lok Sabha then adjourned')

  if endind<startind and endind!=-1:
    endind=-1
    startind=0
    print("emptyy")

  content = content[startind:endind]

  cnt=0
  startindlist = []
  for match in re.finditer(r'[A-Z)] *:', content):
    startindlist.append(match.start())
    cnt+=1

  convlist =[]
  for i in range(len(startindlist)-1):
    convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

  

  convlist1 =[]
  for j,i in enumerate(convlist):
    #print(j)

    i = re.sub('[\(\{\[]I?i?l?nterruptions[\)\}\]]', '', i)
    
    i = removeend(i)

    i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

    i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)
    i = re.sub(r'\d\d\s*\d\d\s*hrs', '', i)

    i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
    i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
    i = re.sub(r'ORAL ANSWERS', ' ', i)
    i = re.sub(r'Oral Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)

    i = re.sub(r'\n', '', i)
    i = re.sub(r' +', ' ', i)
    i = re.sub(r'_+', ' ', i)
    i = re.sub(r'…', ' ', i)
    i = re.sub(r'\.+', ' ', i)

    i = removestart(i)
    if len(i)!=0 and i[-1]!='.':
      i = i + '.'

    if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
    convlist1.append(i)

  convlist6 = []
  ind =0
  for j,i in enumerate(convlist1):
    if len(i)!=0:
      convlist6.append(convlist1[j])
      ind+=1

  if ind==0:
    print("blank csvs")

  dframe = pd.DataFrame(convlist6)  
  print(dframe)   

  #dframe.to_csv('file_name1.csv')
  dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/9/'+str(fid)+ '.csv')

50
                                                     0
0    I beg to lay on the Table:-(1) A copy each of ...
1    I beg to lay on the Table-3 Papers Laid MAY 29...
2    I beg to lay on the Table a co~ of the Report ...
3    On behalf of Shri Nittsh Kumar I beg to lay on...
4    I beg to lay on the Table: (1) A copy of the I...
..                                                 ...
512  On behalf of Shri Devi Lal  I beg to move: IIT...
513  Motion moved: "That the Bill further to amend ...
514  Mr  Deputy Speaker  Sir, this constitution (Am...
515            Mr  Lodha, you may continue tomorrow  .
516                     Sir, I will continue tomorrow.

[517 rows x 1 columns]


## 8th

In [ ]:
def removeend(s):
    se={'(',')','.','\n',' ','\t'}
    k=len(s)-1
    while k!=-1 and (s[k].isupper()==True or s[k] in se):
      if s[k]==')':
        while k!=-1 and s[k]!='(':
          k=k-1
      else:
        k=k-1
    #print(s[i])
    return s[:k+1]

def removestart(s):
    se={' ','-',':','\n'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
#l1=[126,131,133,153,255,307,306,399]
#l2=[132,311]
#for fid in l2:
for fid in range(1,488):
  print(fid)
  raw = parser.from_file('/content/drive/MyDrive/Lok Sabha Data/PDFs/08/' + str(fid) + '.pdf')
  content = raw['content']

  startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
  if startind==-1:
    startind = content.find('MR. SPeAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER in the')
  if startind==-1:
    startind=content.find('MR. SPEAKER ;n the')
  if startind==-1:
    startind = content.find('MR. DEPUTY SPEAKER in the')
  if startind==-1:
    startind=content.find('OBITUARY REFERENCE')  
  if startind==-1:
    startind=content.find('The Lok Sabha met at') 
  if startind==-1:
    startind = content.find('MR. S')
  if startind==-1:
    startind = content.find('MR, S')
  if startind==-1:
    startind = content.find('Mr. S')
  if startind==-1:
    startind = content.find('Mr, S')  
  #if startind==-1:
    #startind = content.find('[S hri C hitta Basu in the')    
  if startind==-1:
    print("empty")
    #empty.append(fid)
    startind = 0
  
  #print(startind)
  endind = content.find('The Lok Sabha then adjourned')

  if endind<startind and endind!=-1:
    endind=-1
    startind=0
    print("emptyy")

  content = content[startind:endind]

  cnt=0
  startindlist = []
  for match in re.finditer(r'[A-Z)] *:', content):
    startindlist.append(match.start())
    cnt+=1

  convlist =[]
  for i in range(len(startindlist)-1):
    convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

  convlist1 =[]
  for j,i in enumerate(convlist):
    #print(j)

    i = re.sub('[\(\{\[]I?i?l?nterruptions[\)\}\]]', '', i)
    
    i = removeend(i)

    i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

    i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)
    i = re.sub(r'\d\d\s*\d\d\s*hrs', '', i)

    i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
    i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
    i = re.sub(r'ORAL ANSWERS', ' ', i)
    i = re.sub(r'Oral Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)
    i = re.sub(r'Written Answers', ' ', i)

    i = re.sub(r'\n', '', i)
    i = re.sub(r' +', ' ', i)
    i = re.sub(r'_+', ' ', i)
    i = re.sub(r'…', ' ', i)
    i = re.sub(r'\.+', ' ', i)

    i = removestart(i)
    if len(i)!=0 and i[-1]!='.':
      i = i + '.'

    if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
    convlist1.append(i)

  convlist6 = []
  ind =0
  for j,i in enumerate(convlist1):
    if len(i)!=0:
      convlist6.append(convlist1[j])
      ind+=1

  if ind==0:
    print("blank csvs")

  dframe = pd.DataFrame(convlist6)  
  #print(dframe)   

  #dframe.to_csv('file_name1.csv')
  dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/8/'+str(fid)+ '.csv')

132
311
blank csvs


## 7th

In [ ]:
empty=[]
def removeend(s):
    se=['.', '।','?','|', '!', '-']
    k=len(s)-1
    while k!=-1 and ((s[k] not in se) or (k!=0 and s[k]=='.' and s[k-1].isupper()==True) and s[k]=='r'):
      #print(s[k])
      k = k-1
    #print(s[k])
    if len(s)-k>30:
      return s
    else:
      return s[:k+1]

def removestart(s):
    se={' ','-',':','\n','|'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(1,466):
  print(fid)
  try:
    raw = parser.from_file('/content/drive/MyDrive/LOK SABHA DATA/07/' + str(fid) + '.pdf')
    content = raw['content']

    startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
    if startind==-1:
      startind = content.find('MR. SPeAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER ;n the')
    if startind==-1:
      startind = content.find('MR. DEPUTY SPEAKER in the')
    if startind==-1:
      startind=content.find('OBITUARY REFERENCE')  
    if startind==-1:
      startind=content.find('The Lok Sabha met at') 
    if startind==-1:
      startind = content.find('MR. S')
    if startind==-1:
      startind = content.find('Mr. S')
    if startind==-1:
      startind = content.find('Mr, S')  
    #if startind==-1:
      #startind = content.find('[S hri C hitta Basu in the')    
    if startind==-1:
      print("empty")
      empty.append(fid)
      startind = 0
    
    endind = content.find('The Lok Sabha then adjourned')

    content = content[startind:endind]

    cnt=0
    startindlist = []
    for match in re.finditer(r':[^-]', content):
      startindlist.append(match.start())
      cnt+=1

    convlist =[]
    for i in range(len(startindlist)-1):
      convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

    convlist1 =[]
    for j,i in enumerate(convlist):
      #print(j)

      i = re.sub('[\(\{\[]\s?I?i?l?nterruptions[\)\}\]]', '', i)
      i = re.sub('\(/nterruptions\)', '', i)
      i = re.sub('(व्यवधान)', '', i)
      
      i = removeend(i)

      i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

      i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)

      i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
      i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
      i = re.sub(r'ORAL ANSWERS', ' ', i)
      i = re.sub(r'Oral Answers', ' ', i)
      i = re.sub(r'Written Answers', ' ', i)
      i = re.sub(r'To Questions', ' ', i)

      i = re.sub(r'\n', '', i)
      i = re.sub(r' +', ' ', i)
      i = re.sub(r'_+', ' ', i)
      i = re.sub(r'\.+', ' ', i)
      i = re.sub(r'…', ' ', i)
      i = removestart(i)
      
      if len(i)!=0 and (i[-1]!='.' or i[-1]!='|' or i[-1]!='।'):
        if len(i)>2 and ord(i[-2])>2000 and ord(i[-2])<3000:
          i=i+'।'
        else:
          i = i + '.'

      if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
      convlist1.append(i)

    convlist6 = []
    ind =0
    for j,i in enumerate(convlist1):
      if len(i)!=0 :
        convlist6.append(convlist1[j])
        ind+=1
    dframe = pd.DataFrame(convlist6)  
    #print(dframe)   

    #dframe.to_csv('file_name1.csv')
    dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/7/'+str(fid)+ '.csv')
  except:
    print("error")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## 6th

In [ ]:
empty=[]
def removeend(s):
    se=['.', '।','?','|', '!', '-']
    k=len(s)-1
    while k!=-1 and ((s[k] not in se) or (k!=0 and s[k]=='.' and s[k-1].isupper()==True) and s[k]=='r'):
      #print(s[k])
      k = k-1
    #print(s[k])
    if len(s)-k>30:
      return s
    else:
      return s[:k+1]

def removestart(s):
    se={' ','-',':','\n','|'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(1,268):
  print(fid)
  try:
    raw = parser.from_file('/content/drive/MyDrive/LOK SABHA DATA/06/' + str(fid) + '.pdf')
    content = raw['content']

    startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
    if startind==-1:
      startind = content.find('MR. SPeAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER ;n the')
    if startind==-1:
      startind = content.find('MR. DEPUTY SPEAKER in the')
    if startind==-1:
      startind=content.find('OBITUARY REFERENCE')  
    if startind==-1:
      startind=content.find('The Lok Sabha met at') 
    if startind==-1:
      startind = content.find('MR. S')
    if startind==-1:
      startind = content.find('Mr. S')
    if startind==-1:
      startind = content.find('Mr, S')  
    #if startind==-1:
      #startind = content.find('[S hri C hitta Basu in the')    
    if startind==-1:
      print("empty")
      empty.append(fid)
      startind = 0
    
    endind = content.find('The Lok Sabha then adjourned')

    content = content[startind:endind]

    cnt=0
    startindlist = []
    for match in re.finditer(r':[^-]', content):
      startindlist.append(match.start())
      cnt+=1

    convlist =[]
    for i in range(len(startindlist)-1):
      convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

    convlist1 =[]
    for j,i in enumerate(convlist):
      #print(j)

      i = re.sub('[\(\{\[]\s?I?i?l?nterruptions[\)\}\]]', '', i)
      i = re.sub('\(/nterruptions\)', '', i)
      i = re.sub('(व्यवधान)', '', i)
      
      i = removeend(i)

      i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

      i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)

      i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
      i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
      i = re.sub(r'ORAL ANSWERS', ' ', i)
      i = re.sub(r'Oral Answers', ' ', i)
      i = re.sub(r'Written Answers', ' ', i)
      i = re.sub(r'To Questions', ' ', i)

      i = re.sub(r'\n', '', i)
      i = re.sub(r' +', ' ', i)
      i = re.sub(r'_+', ' ', i)
      i = re.sub(r'\.+', ' ', i)
      i = re.sub(r'…', ' ', i)
      i = removestart(i)
      
      if len(i)!=0 and (i[-1]!='.' or i[-1]!='|' or i[-1]!='।'):
        if len(i)>2 and ord(i[-2])>2000 and ord(i[-2])<3000:
          i=i+'।'
        else:
          i = i + '.'

      if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
      convlist1.append(i)

    convlist6 = []
    ind =0
    for j,i in enumerate(convlist1):
      if len(i)!=0 :
        convlist6.append(convlist1[j])
        ind+=1
    dframe = pd.DataFrame(convlist6)  
    #print(dframe)   

    #dframe.to_csv('file_name1.csv')
    dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/6/'+str(fid)+ '.csv')
  except:
    print("error")


1
2
3
error
4
5
6
7
8
error
9
10
11
12
13
14
15
16
17
18
19
20
error
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
error
36
37
38
error
39
40
41
42
43
44
45
46
47
48
error
49
50
51
52
53
54
55
56
57
58
59
60
error
61
62
63
64
error
65
66
67
68
69
70
71
72
73
74
75
76
error
77
78
79
80
81
82
empty
83
84
85
86
87
error
88
error
89
error
90
91
92
93
94
error
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
error
113
114
error
115
116
error
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
error
133
134
135
136
137
138
139
error
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
error
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
error
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
error
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244


## 5th

In [ ]:
empty=[]
def removeend(s):
    se=['.', '।','?','|', '!', '-']
    k=len(s)-1
    while k!=-1 and ((s[k] not in se) or (k!=0 and s[k]=='.' and s[k-1].isupper()==True) and s[k]=='r'):
      #print(s[k])
      k = k-1
    #print(s[k])
    if len(s)-k>30:
      return s
    else:
      return s[:k+1]

def removestart(s):
    se={' ','-',':','\n','|'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(1,615):
  print(fid)
  try:
    raw = parser.from_file('/content/drive/MyDrive/LOK SABHA DATA/05/' + str(fid) + '.pdf')
    content = raw['content']

    startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
    if startind==-1:
      startind = content.find('MR. SPeAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER ;n the')
    if startind==-1:
      startind = content.find('MR. DEPUTY SPEAKER in the')
    if startind==-1:
      startind=content.find('OBITUARY REFERENCE')  
    if startind==-1:
      startind=content.find('The Lok Sabha met at') 
    if startind==-1:
      startind = content.find('MR. S')
    if startind==-1:
      startind = content.find('Mr. S')
    if startind==-1:
      startind = content.find('Mr, S')  
    #if startind==-1:
      #startind = content.find('[S hri C hitta Basu in the')    
    if startind==-1:
      print("empty")
      empty.append(fid)
      startind = 0
    
    endind = content.find('The Lok Sabha then adjourned')

    content = content[startind:endind]

    cnt=0
    startindlist = []
    for match in re.finditer(r':[^-]', content):
      startindlist.append(match.start())
      cnt+=1

    convlist =[]
    for i in range(len(startindlist)-1):
      convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

    convlist1 =[]
    for j,i in enumerate(convlist):
      #print(j)

      i = re.sub('[\(\{\[]\s?I?i?l?nterruptions[\)\}\]]', '', i)
      i = re.sub('\(/nterruptions\)', '', i)
      i = re.sub('(व्यवधान)', '', i)
      
      i = removeend(i)

      i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

      i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)

      i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
      i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
      i = re.sub(r'ORAL ANSWERS', ' ', i)
      i = re.sub(r'Oral Answers', ' ', i)
      i = re.sub(r'Written Answers', ' ', i)
      i = re.sub(r'To Questions', ' ', i)

      i = re.sub(r'\n', '', i)
      i = re.sub(r' +', ' ', i)
      i = re.sub(r'_+', ' ', i)
      i = re.sub(r'\.+', ' ', i)
      i = re.sub(r'…', ' ', i)
      i = removestart(i)
      
      if len(i)!=0 and (i[-1]!='.' or i[-1]!='|' or i[-1]!='।'):
        if ord(i[-2])>2000 and ord(i[-2])<3000:
          i=i+'।'
        else:
          i = i + '.'

      if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
      convlist1.append(i)

    convlist6 = []
    ind =0
    for j,i in enumerate(convlist1):
      if len(i)!=0 :
        convlist6.append(convlist1[j])
        ind+=1
    dframe = pd.DataFrame(convlist6)  
    #print(dframe)   

    #dframe.to_csv('file_name1.csv')
    dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/5/'+str(fid)+ '.csv')
  except:
    print("error")


1
2
3
4
5
6
7
8
error
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
error
106
107
108
109
110
111
112
113
114
error
115
116
117
118
119
120
121
122
error
123
124
125
126
127
128
129
130
131
error
132
133
134
error
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
error
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
error
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
error
260
261
error
262


## 4th 

In [ ]:
empty=[]
def removeend(s):
    se=['.', '।','?','|', '!', '-']
    k=len(s)-1
    while k!=-1 and ((s[k] not in se) or (k!=0 and s[k]=='.' and s[k-1].isupper()==True) and s[k]=='r'):
      #print(s[k])
      k = k-1
    #print(s[k])
    if len(s)-k>30:
      return s
    else:
      return s[:k+1]

def removestart(s):
    se={' ','-',':','\n','|'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(1,469):
  print(fid)
  try:
    raw = parser.from_file('/content/drive/MyDrive/LOK SABHA DATA/04/' + str(fid) + '.pdf')
    content = raw['content']

    startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
    if startind==-1:
      startind = content.find('MR. SPeAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER ;n the')
    if startind==-1:
      startind = content.find('MR. DEPUTY SPEAKER in the')
    if startind==-1:
      startind=content.find('OBITUARY REFERENCE')  
    if startind==-1:
      startind=content.find('The Lok Sabha met at') 
    if startind==-1:
      startind = content.find('MR. S')
    if startind==-1:
      startind = content.find('Mr. S')
    if startind==-1:
      startind = content.find('Mr, S')  
    #if startind==-1:
      #startind = content.find('[S hri C hitta Basu in the')    
    if startind==-1:
      print("empty")
      empty.append(fid)
      startind = 0
    
    endind = content.find('The Lok Sabha then adjourned')

    content = content[startind:endind]

    cnt=0
    startindlist = []
    for match in re.finditer(r':[^-]', content):
      startindlist.append(match.start())
      cnt+=1

    convlist =[]
    for i in range(len(startindlist)-1):
      convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

    convlist1 =[]
    for j,i in enumerate(convlist):
      #print(j)

      i = re.sub('[\(\{\[]\s?I?i?l?nterruptions[\)\}\]]', '', i)
      i = re.sub('\(/nterruptions\)', '', i)
      i = re.sub('(व्यवधान)', '', i)
      
      i = removeend(i)

      i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

      i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)

      i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
      i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
      i = re.sub(r'ORAL ANSWERS', ' ', i)
      i = re.sub(r'Oral Answers', ' ', i)
      i = re.sub(r'Written Answers', ' ', i)
      i = re.sub(r'To Questions', ' ', i)

      i = re.sub(r'\n', '', i)
      i = re.sub(r' +', ' ', i)
      i = re.sub(r'_+', ' ', i)
      i = re.sub(r'\.+', ' ', i)
      i = re.sub(r'…', ' ', i)
      i = removestart(i)
      
      if len(i)!=0 and (i[-1]!='.' or i[-1]!='|' or i[-1]!='।'):
        if ord(i[-1])>2000 and ord(i[-1])<3000:
          i=i+'।'
        else:
          i = i + '.'

      if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
      convlist1.append(i)

    convlist6 = []
    ind =0
    for j,i in enumerate(convlist1):
      if len(i)!=0 :
        convlist6.append(convlist1[j])
        ind+=1
    dframe = pd.DataFrame(convlist6)  
    #print(dframe)   

    #dframe.to_csv('file_name1.csv')
    dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/4/'+str(fid)+ '.csv')
  except:
    print("error")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## 3th

In [ ]:
empty=[]
def removeend(s):
    se=['.', '।','?','|', '!', '-']
    k=len(s)-1
    while k!=-1 and ((s[k] not in se) or (k!=0 and s[k]=='.' and s[k-1].isupper()==True) and s[k]=='r'):
      #print(s[k])
      k = k-1
    #print(s[k])
    if len(s)-k>30:
      return s
    else:
      return s[:k+1]

def removestart(s):
    se={' ','-',':','\n','|'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(1,579):
  print(fid)
  try:
    raw = parser.from_file('/content/drive/MyDrive/LOK SABHA DATA/03/' + str(fid) + '.pdf')
    content = raw['content']

    startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
    if startind==-1:
      startind = content.find('MR. SPeAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER ;n the')
    if startind==-1:
      startind = content.find('MR. DEPUTY SPEAKER in the')
    if startind==-1:
      startind=content.find('OBITUARY REFERENCE')  
    if startind==-1:
      startind=content.find('The Lok Sabha met at') 
    if startind==-1:
      startind = content.find('MR. S')
    if startind==-1:
      startind = content.find('Mr. S')
    if startind==-1:
      startind = content.find('Mr, S')  
    #if startind==-1:
      #startind = content.find('[S hri C hitta Basu in the')    
    if startind==-1:
      print("empty")
      empty.append(fid)
      startind = 0
    
    endind = content.find('The Lok Sabha then adjourned')

    content = content[startind:endind]

    cnt=0
    startindlist = []
    for match in re.finditer(r':[^-]', content):
      startindlist.append(match.start())
      cnt+=1

    convlist =[]
    for i in range(len(startindlist)-1):
      convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

    convlist1 =[]
    for j,i in enumerate(convlist):
      #print(j)

      i = re.sub('[\(\{\[]\s?I?i?l?nterruptions[\)\}\]]', '', i)
      i = re.sub('\(/nterruptions\)', '', i)
      i = re.sub('(व्यवधान)', '', i)
      
      i = removeend(i)

      i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

      i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)

      i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
      i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
      i = re.sub(r'ORAL ANSWERS', ' ', i)
      i = re.sub(r'Oral Answers', ' ', i)
      i = re.sub(r'Written Answers', ' ', i)
      i = re.sub(r'To Questions', ' ', i)

      i = re.sub(r'\n', '', i)
      i = re.sub(r' +', ' ', i)
      i = re.sub(r'_+', ' ', i)
      i = re.sub(r'\.+', ' ', i)
      i = re.sub(r'…', ' ', i)
      i = removestart(i)
      
      if len(i)!=0 and (i[-1]!='.' or i[-1]!='|' or i[-1]!='।'):
        if ord(i[-1])>2000 and ord(i[-1])<3000:
          i=i+'।'
        else:
          i = i + '.'

      if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
      convlist1.append(i)

    convlist6 = []
    ind =0
    for j,i in enumerate(convlist1):
      if len(i)!=0 :
        convlist6.append(convlist1[j])
        ind+=1
    dframe = pd.DataFrame(convlist6)  
    #print(dframe)   

    #dframe.to_csv('file_name1.csv')
    dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/3/'+str(fid)+ '.csv')
  except:
    print("error")


1


2021-10-14 08:13:09,230 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2021-10-14 08:13:09,973 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2021-10-14 08:13:10,438 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
error
269
270
271
272
273
274
275
276


## 2th

In [ ]:
empty=[]
def removeend(s):
    se=['.', '।','?','|', '!', '-']
    k=len(s)-1
    while k!=-1 and ((s[k] not in se) or (k!=0 and s[k]=='.' and s[k-1].isupper()==True) and s[k]=='r'):
      #print(s[k])
      k = k-1
    #print(s[k])
    if len(s)-k>30:
      return s
    else:
      return s[:k+1]

def removestart(s):
    se={' ','-',':','\n','|'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(1,15):
  print(fid)
  try:
    raw = parser.from_file('/content/drive/MyDrive/LOK SABHA DATA/02/' + str(fid) + '.pdf')
    content = raw['content']

    startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
    if startind==-1:
      startind = content.find('MR. SPeAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER ;n the')
    if startind==-1:
      startind = content.find('MR. DEPUTY SPEAKER in the')
    if startind==-1:
      startind=content.find('OBITUARY REFERENCE')  
    if startind==-1:
      startind=content.find('The Lok Sabha met at') 
    if startind==-1:
      startind = content.find('MR. S')
    if startind==-1:
      startind = content.find('Mr. S')
    if startind==-1:
      startind = content.find('Mr, S')  
    #if startind==-1:
      #startind = content.find('[S hri C hitta Basu in the')    
    if startind==-1:
      print("empty")
      empty.append(fid)
      startind = 0
    
    endind = content.find('The Lok Sabha then adjourned')

    content = content[startind:endind]

    cnt=0
    startindlist = []
    for match in re.finditer(r':[^-]', content):
      startindlist.append(match.start())
      cnt+=1

    convlist =[]
    for i in range(len(startindlist)-1):
      convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

    convlist1 =[]
    for j,i in enumerate(convlist):
      #print(j)

      i = re.sub('[\(\{\[]\s?I?i?l?nterruptions[\)\}\]]', '', i)
      i = re.sub('\(/nterruptions\)', '', i)
      i = re.sub('(व्यवधान)', '', i)
      
      i = removeend(i)

      i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

      i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)

      i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
      i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
      i = re.sub(r'ORAL ANSWERS', ' ', i)
      i = re.sub(r'Oral Answers', ' ', i)
      i = re.sub(r'Written Answers', ' ', i)
      i = re.sub(r'To Questions', ' ', i)

      i = re.sub(r'\n', '', i)
      i = re.sub(r' +', ' ', i)
      i = re.sub(r'_+', ' ', i)
      i = re.sub(r'\.+', ' ', i)
      i = re.sub(r'…', ' ', i)
      i = removestart(i)
      
      if len(i)!=0 and (i[-1]!='.' or i[-1]!='|' or i[-1]!='।'):
        if len(i)>2 and ord(i[-2])>2000 and ord(i[-2])<3000::
          i=i+'।'
        else:
          i = i + '.'

      if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
      convlist1.append(i)

    convlist6 = []
    ind =0
    for j,i in enumerate(convlist1):
      if len(i)!=0 :
        convlist6.append(convlist1[j])
        ind+=1
    dframe = pd.DataFrame(convlist6)  
    #print(dframe)   

    #dframe.to_csv('file_name1.csv')
    dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/2/'+str(fid)+ '.csv')
  except:
    print("error")


1
error
2
error
3
error
4
error
5
error
6
error
7
error
8
error
9
error
10
error
11
error
12
error
13
error
14
error
15
error
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
empty
error
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
25

## 1st

In [ ]:
empty=[]
def removeend(s):
    se=['.', '।','?','|', '!', '-']
    k=len(s)-1
    while k!=-1 and ((s[k] not in se) or (k!=0 and s[k]=='.' and s[k-1].isupper()==True) and s[k]=='r'):
      #print(s[k])
      k = k-1
    #print(s[k])
    if len(s)-k>30:
      return s
    else:
      return s[:k+1]

def removestart(s):
    se={' ','-',':','\n','|'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

content=''
for fid in range(1,15):
  print(fid)
  try:
    raw = parser.from_file('/content/drive/MyDrive/LOK SABHA DATA/01/' + str(fid) + '.pdf')
    content = raw['content']

    startind = content.find('MR. SPEAKER in the') #[Mr. Speaktr in the Chair] #[Mr SPEAKER in the Chair].{Mr, SPEAKER ‘in the Chair][Mr. SPEAKER in the Chair] [Mr, SPEAKER in the Chair]     #[MR. SPeAKER in the Chair). #(MR. SPEAKER in the Chain      #[MR. SPEAKER ;n the Cha;r)    #[MR. DEPUTY SPEAKER.  #[MR. SPEAKER.    #(Mil. SPEAKER In the Chair]      #[M r. S peaker in the Chaii]
    if startind==-1:
      startind = content.find('MR. SPeAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER in the')
    if startind==-1:
      startind=content.find('MR. SPEAKER ;n the')
    if startind==-1:
      startind = content.find('MR. DEPUTY SPEAKER in the')
    if startind==-1:
      startind=content.find('OBITUARY REFERENCE')  
    if startind==-1:
      startind=content.find('The Lok Sabha met at') 
    if startind==-1:
      startind = content.find('MR. S')
    if startind==-1:
      startind = content.find('Mr. S')
    if startind==-1:
      startind = content.find('Mr, S')  
    #if startind==-1:
      #startind = content.find('[S hri C hitta Basu in the')    
    if startind==-1:
      print("empty")
      empty.append(fid)
      startind = 0
    
    endind = content.find('The Lok Sabha then adjourned')

    content = content[startind:endind]

    cnt=0
    startindlist = []
    for match in re.finditer(r':[^-]', content):
      startindlist.append(match.start())
      cnt+=1

    convlist =[]
    for i in range(len(startindlist)-1):
      convlist.append(content[startindlist[i]+1 : startindlist[i+1]])

    convlist1 =[]
    for j,i in enumerate(convlist):
      #print(j)

      i = re.sub('[\(\{\[]\s?I?i?l?nterruptions[\)\}\]]', '', i)
      i = re.sub('\(/nterruptions\)', '', i)
      i = re.sub('(व्यवधान)', '', i)
      
      i = removeend(i)

      i =  re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i)

      i = re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i)

      i = re.sub(r'[\(\{\[]Translation[\)\}\]]', ' ', i)
      i = re.sub(r'[\(\{\[]English[\)\}\]]', ' ', i)
      i = re.sub(r'ORAL ANSWERS', ' ', i)
      i = re.sub(r'Oral Answers', ' ', i)
      i = re.sub(r'Written Answers', ' ', i)
      i = re.sub(r'To Questions', ' ', i)

      i = re.sub(r'\n', '', i)
      i = re.sub(r' +', ' ', i)
      i = re.sub(r'_+', ' ', i)
      i = re.sub(r'\.+', ' ', i)
      i = re.sub(r'…', ' ', i)
      i = removestart(i)
      
      if len(i)!=0 and (i[-1]!='.' or i[-1]!='|' or i[-1]!='।'):
        if len(i)>2 and ord(i[-2])>2000 and ord(i[-2])<3000:
          i=i+'।'
        else:
          i = i + '.'

      if (i[:3]!='Shri' or i[:1]!='Dr') and len(i)<15:
        i=''
      convlist1.append(i)

    convlist6 = []
    ind =0
    for j,i in enumerate(convlist1):
      if len(i)!=0 :
        convlist6.append(convlist1[j])
        ind+=1
    dframe = pd.DataFrame(convlist6)  
    #print(dframe)   

    #dframe.to_csv('file_name1.csv')
    dframe.to_csv('/content/drive/MyDrive/Lok Sabha Data/textfiles/1/'+str(fid)+ '.csv')
  except:
    print("error")


15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
2

# For a new session

In [ ]:
# a dummy to read text column-wise from the pdfs.
raw = parser.from_file('/content/drive/MyDrive/LOK SABHA DATA/07/464.pdf')

content = raw['content']    # returns entire text as a string, sufficing the column-wise read

In [ ]:
print(content)



















































Seventh Series, Vol. I, No. 2

LOK SABHA

DEBATES

First Session
(Seventh Lok Sabha)

   
सत्यमेव जयते

(Vol. I Contains No.1 to 10)

LOK SABHA SECRETARIAT
NEW DELHI
Price: Rs., 06.00

Tuesday, January 22, 1980

Magha02, 1901(Saka)



CONTENTS

Ne. 2, Tuesday, January 22, 1980/Magha 2, 100०1 (Saka)

CoLUMNS

Members sworn. . . . . . eee . ० 1--3

Election of the Speaker . - - * . . | 3-4

Felicitations to the Speaker. . . . . . 4

Shrimati Indira Gandhi. . . . «9 . - 4-९

Shri Charan Singh 5-6

Shri Samar Mukherjee 6-7

Shri Ravindra Varma 7—9

Shri C.T. Dhandapani 9-10

Shri Yeshwantrao Chavan . IO-11

Shri Indrajit Gupta 1I—I3

Shri Tridib Chaudhuri 13

Shri Ebrahim Sulaiman Sait 13-14

Shri Chitta Basu I4-I§

Dr. Faroog Abdullah 15-16

Shri N. Soundararajan 16-17

Shri Frank Anthony 17—20

Mr. Speaker 21--24





LOK SABHA DEBATES
 

we ee ee

LOK SABHA

 

Tuesday January 22, 1980/Magha 2

1901 (Saka)

 

The Lok Sabha met at Eleven

In [ ]:
for i in content:
  if ord(i)>2000 and ord(i)<3000:
    print(i) 

In [ ]:
l =re.findall('in the Chair', content)
print(l)


['in the Chair', 'in the Chair', 'in the Chair', 'in the Chair', 'in the Chair', 'in the Chair']


In [ ]:
startind = re.search(content,'MR. SPEAKER')
startind

In [ ]:
# !pip install PyMuPDF
# !pip install easyocr
# import fitz
# doc = fitz.open('/content/drive/MyDrive/Lok Sabha Data/08/372.pdf')
# page = doc[1]

# print(page.get_text())

In [ ]:
page.get_text()

''

In [ ]:
raw.keys()

dict_keys(['metadata', 'content', 'status'])

In [ ]:
startind = content.find('[HON. SPEAKER')
startind

5769

In [ ]:
# if startind==-1:
#   startind=content.find('MR. SPEAKER')
# startind

-1

In [ ]:
content= content[startind:]
content

"[HON. SPEAKER  in the Chair] \n\n \n\n \n\n     \n\n\n\n04.02.2019                                                                                                                     8 \n\n \n… (Interruptions) \n\nPROF. SAUGATA ROY (DUM DUM): Madam, I have given a notice for \n\nAdjournment Motion. There is a constitutional crisis. …(Interruptions) The \n\nChief Minister of West Bengal is sitting on a dharna. …(Interruptions) \n\nमाननीय अ�य� :एक िमनट ।    \n\n…(�यवधान) \n\n  \n\n\n\n04.02.2019                                                                                                                     9 \n\n \n11 01 hrs  \n\nOBITUARY REFERENCES \n\nमाननीय अ�य� :माननीय सद�यगण, मझेु सभा को आठ पूव� सद�य�, सव��ी केयूर भूषण, \n\nबटे�र हेमराम, वाई. जी. महाजन, डॉ. जी. आर. सरोद,े जलागम क�डला राव, के. कृ�णमूित�, भान ु\n\n�काश िसंह और जॉज� फना�डीज के दःुखद िनधन के बार ेम� सूिचत करना है।  \n\n �ी केयूर भूषण म�य �देश के रायपरु संसदीय िनवा�चन �े�, जो अब छ�ीसगढ़ म� है, से \n\n7व� और 8व� लोक सभ

In [ ]:
content

"[HON. SPEAKER  in the Chair] \n\n \n\n \n\n     \n\n\n\n04.02.2019                                                                                                                     8 \n\n \n… (Interruptions) \n\nPROF. SAUGATA ROY (DUM DUM): Madam, I have given a notice for \n\nAdjournment Motion. There is a constitutional crisis. …(Interruptions) The \n\nChief Minister of West Bengal is sitting on a dharna. …(Interruptions) \n\nमाननीय अ�य� :एक िमनट ।    \n\n…(�यवधान) \n\n  \n\n\n\n04.02.2019                                                                                                                     9 \n\n \n11 01 hrs  \n\nOBITUARY REFERENCES \n\nमाननीय अ�य� :माननीय सद�यगण, मझेु सभा को आठ पूव� सद�य�, सव��ी केयूर भूषण, \n\nबटे�र हेमराम, वाई. जी. महाजन, डॉ. जी. आर. सरोद,े जलागम क�डला राव, के. कृ�णमूित�, भान ु\n\n�काश िसंह और जॉज� फना�डीज के दःुखद िनधन के बार ेम� सूिचत करना है।  \n\n �ी केयूर भूषण म�य �देश के रायपरु संसदीय िनवा�चन �े�, जो अब छ�ीसगढ़ म� है, से \n\n7व� और 8व� लोक सभ

In [ ]:
i=0
startindlist = []
for match in re.finditer(r':[^-]', content):
  startindlist.append(match.start())
  i+=1

print(i)

startindlist.append(len(content))

convlist =[]
for i in range(len(startindlist)-1):
  convlist.append(content[startindlist[i]-20 : startindlist[i+1]])

#convlist

5


In [ ]:
convlist

['SAUGATA ROY (DUM DUM): Madam, I have given a notice for \n\nAdjournment Motion. There is a constitutional crisis. …(Interruptions) The \n\nChief Minister of West Bengal is sitting on a dharna. …(Interruptions) \n\nमाननीय अ�य� :एक िमनट ।    \n\n…(�यवधान) \n\n  \n\n\n\n04.02.2019                                                                                                                     9 \n\n \n11 01 hrs  \n\nOBITUARY REFERENCES \n\nमाननीय अ�य� :माननीय सद�यगण, मझेु सभा को आठ पूव� सद�य�, सव��ी केयूर भूषण, \n\nबटे�र हेमराम, वाई. जी. महाजन, डॉ. जी. आर. सरोद,े जलागम क�डला राव, के. कृ�णमूित�, भान ु\n\n�काश िसंह और जॉज� फना�डीज के दःुखद िनधन के बार ेम� सूिचत करना है।  \n\n �ी केयूर भूषण म�य �देश के रायपरु संसदीय िनवा�चन �े�, जो अब छ�ीसगढ़ म� है, से \n\n7व� और 8व� लोक सभा के सद�य थे।  \n\n �ी केयूर भूषण �ा�कलन सिमित के सद�य तथा मानिसक �वा��य िवधेयक सबंंधी \n\nसयं�ु ससंदीय सिमित के सद�य थे। \n\n �ी भूषण �वतं�ता सेनानी थे िज�ह� ‘भारत छोड़ो’ आंदोलन म� भाग लेन ेके िलए कारावास \n\nम� रखा गया

In [ ]:
#printing reverse string for better visualization
# for i in convlist:
#   print(repr(i[::-1]))      #\\\\\\\\\\\\\\\।

In [ ]:
for j,i in enumerate(convlist):
  convlist[j] = re.sub('\(Interruptions\)', '', i)
  convlist[j] = re.sub('\(Interruptions\)', '', i)
  #(Interruptions)

In [ ]:
convlist1 = []
def removeend(s):
    se={'(',')','.','\n',' '}
    k=len(s)-1
    while k!=-1 and (s[k].isupper()==True or s[k] in se):
      if s[k]==')':
        while k!=-1 and s[k]!='(':
          k=k-1
      else:
        k=k-1
    #print(s[i])
    return s[:k+1]

def removestart(s):
    se={' ','-',':','\n'}
    k=0
    while k!=len(s) and s[k] in se:
      k=k+1
    #print(s[i])
    return s[k:]

for s in convlist:
  s = removeend(s)
  s = removestart(s)
  convlist1.append(s)

In [ ]:
convlist2=[]
for i in convlist1:
  convlist2.append(re.sub(r'\[*\(*P?p?laced in L?l?ibrary. See No. L?l? T?t? \d*\d\d\d\d*/*\d\d\)*\]*', '', i))
  #[placed in Library, See No. LT 104471091

In [ ]:
# convlist3=[]
# for i in convlist2:
#   i = re.sub(r'\d\d\*.\d\d\*.20\d\d', '', i)
#   convlist3.append(i)

In [ ]:
convlist4=[]
for i in convlist3:
  convlist4.append(re.sub(r'\d\d\.\d\d\s*hrs\.?', '', i))
  

In [ ]:
convlist4

In [ ]:
convlist5=[]
for i in convlist4:
  i = re.sub(r'\n', '', i)
  i = re.sub(r' +', ' ', i)
  i = re.sub(r'_+', ' ', i)
  i = re.sub(r'\.+', ' ', i)
  i = re.sub(r'…', ' ', i)
  convlist5.append(i)

In [ ]:
convlist5[-1]

In [ ]:
convlist5

In [ ]:
for j,i in enumerate(convlist5):
  if len(i)!=0 and i[-1]!='.':
    convlist5[j] = i + '.'

convlist6 = []
ind =0
for j,i in enumerate(convlist5):
  if len(i)!=0:
    convlist6.append(convlist5[j])
    ind+=1
  
convlist6 = convlist6[:-1]    

In [ ]:
len(convlist6)

1001

In [ ]:
import pandas as pd  

In [ ]:
dframe = pd.DataFrame(convlist6)  
print(dframe)   

                                                      0
0     Shri Asaduddin Owaisi, Q  No  442  World Bank ...
1     Will the Minister of FINANCE be pleased to sta...
2     (a) to (e) A Statement is placed on the Table ...
3     Mr  Speaker, Sir, part (a) of my question is t...
4     I am not able to understand the question  3 Or...
...                                                 ...
996   Sir, I beg to move: "That the Bill, as amended...
997   The question is: "That the Bill, as amended, b...
998   I think, I have beaten the Rajdhani Expressl V...
999   Hon  Members, thb ;sixth Session of the Fourte...
1000  The hon  Members may now stand up as Vande Mat...

[1001 rows x 1 columns]


In [ ]:
dframe.to_csv('file_name.csv')

In [ ]:
# import csv
# file = open('/content/drive/MyDrive/Lok Sabha Data/textfiles/16/1.csv')
# csvreader = csv.reader(file)
# rows = []
# for row in csvreader:
#         rows.append(row)
# rows

# Fixing I, purna-viram glitch

In [ ]:
import csv
import re

file = open('/content/drive/MyDrive/Lok Sabha Data/textfiles/16/4.csv')
csvreader = csv.reader(file)
rows = []
for j,row in enumerate(csvreader):
  rows.append(row)

# for i,row in enumerate(rows):
#   try:
#     x = row[1]
#   except:
#     print(row)
#     print(i)
#     break


for i, row in enumerate(rows):
  indexes = list()
  for k,char in enumerate(row[1]):
    if char == '|':
      indexes.append(k)

  for index in indexes:
    temp = index-1
    
    while(temp>-1 and row[1][temp]==' '):
      temp -= 1

    if temp!=-1:
      if(ord(rows[i][1][temp])>2000 and rows[i][1][temp]!='।'):
        rows[i][1] = rows[i][1][:index] + '।' + rows[i][1][index+1:]
      
      else:
        rows[i][1] = rows[i][1][:index] + 'I' + rows[i][1][index+1:]
    
    else:
      rows[i][1] = rows[i][1][:index] + 'I' + rows[i][1][index+1:]


dframe = pd.DataFrame(rows)
dframe.to_csv('/content/temp.csv')

In [ ]:
print(indexes)

[]


In [ ]:
print(type(row[1]))

<class 'str'>


In [ ]:
import re
import os
import csv
import pandas as pd

folderNumber = "16"
folderPath = "/content/drive/MyDrive/Lok Sabha Data/textfiles/"

for curr in range(1,74):
  
  file = open(folderPath + folderNumber + "/" + str(curr) + ".csv")
  csvreader = csv.reader(file)
  rows = []

  for j,row in enumerate(csvreader):
    rows.append(row)

  for i, row in enumerate(rows):
    indexes = list()
    if len(row)!=2:
      print(len(row))
    for k,char in enumerate(row[1]):
      if char == '|':
        indexes.append(k)

    for index in indexes:
      temp = index-1
      
      while(temp>-1 and row[1][temp]==' '):
        temp -= 1

      if temp!=-1:
        if(ord(rows[i][1][temp])>2000 and rows[i][1][temp]!='।'):
          rows[i][1] = rows[i][1][:index] + '।' + rows[i][1][index+1:]
        
        else:
          rows[i][1] = rows[i][1][:index] + 'I' + rows[i][1][index+1:]
      
      else:
        rows[i][1] = rows[i][1][:index] + 'I' + rows[i][1][index+1:]


  dframe = pd.DataFrame(rows)
  dframe.to_csv(folderPath + folderNumber + "/" + str(curr) + ".csv")
  print(f"{curr} Completed")

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1 Completed
2 Completed
3 Completed
4 Completed
5 Completed
6 Completed
7 Completed
8 Completed
9 Completed
10 Completed
11 Completed
12 Completed
13 Completed
14 Completed
15 Completed
16 Completed
17 Completed
18 Completed
19 Completed
20 Completed
21 Completed
22 Completed
23 Completed
24 Completed
25 Completed
26 Completed
27 Completed
28 Completed
29 Completed
30 Completed
31 Completed
32 Completed
33 Completed
34 Completed
35 Completed
36 Completed
37 Completed
38 Completed
39 Completed
40 Completed
41 Completed
42 Completed
43 Completed
44 Completed
45 Completed
46 Completed
47 Completed
48 Completed
49 Completed
50 Completed
51 Completed
52 Completed
53 Completed
54 Completed
55 Completed
56 Completed
57 Completed
58 Completed
59 Completed
60 Completed
61 Completed
62 Completed
63 Completed
64 Completed
65 Completed
66 Completed
67 Completed
68 Completed
69 Completed
70 Completed
71 Completed
72 Completed
73 Completed
